# RMG model sanity check

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from rmgpy.chemkin import load_chemkin_file

%matplotlib inline


def plot_thermo(Ts, thermo):
    """
    Plot the thermo for a thermo data object at the given temperatures.
    """
    enthalpies, entropies, cps = np.zeros_like(Ts), np.zeros_like(Ts), np.zeros_like(Ts)
    for i, T in enumerate(Ts):
        enthalpies[i] = thermo.get_enthalpy(T)
        entropies[i] = thermo.get_entropy(T)
        cps[i] = thermo.get_heat_capacity(T)

    fig, axes = plt.subplots(1,3, figsize=(14,4))

    axes[0].plot(Ts, enthalpies/4184, label='enthalpies')
    axes[0].set(xlabel='T [K]', ylabel='H [kcal/mol]')
    axes[0].hlines(y=0, xmin=np.min(Ts), xmax=np.max(Ts), linestyle='--', color='r')
    axes[0].legend(fontsize=10)

    axes[1].plot(Ts, entropies/4.184, label='entropy')
    axes[1].set(xlabel='T [K]', ylabel='S [cal/mol/K]')
    axes[1].hlines(y=0, xmin=np.min(Ts), xmax=np.max(Ts), linestyle='--', color='r')
    axes[1].legend(fontsize=10)

    axes[2].plot(Ts, cps/8.314, label='Cp/R')
    axes[2].set(xlabel='T [K]', ylabel='Cp/R')
    axes[2].hlines(y=0, xmin=np.min(Ts), xmax=np.max(Ts), linestyle='--', color='r')
    axes[2].legend()

## 1. Read RMG model

In [2]:
spcs, rxns = load_chemkin_file(
                path='/Users/xiaorui/Dropbox (Personal)/RMG/Co-OPTIMA/Butyl acetate/RMG_model/nBA/nBA_5/chem_annotated.inp',
                dictionary_path='/Users/xiaorui/Dropbox (Personal)/RMG/Co-OPTIMA/Butyl acetate/RMG_model/nBA/nBA_5/species_dictionary.txt',)

spc_dict = {spc.label: spc for spc in spcs}
rxn_dict = {rxn.index: rxn for rxn in rxns}

## 2. Check Heat capacity and entropy values

### 2.1 Positivity
We expect that heat capacities and entropies to be non-negative. This should be true at least for the applicable (fitted) range. Check passes if no output returned

In [13]:
default_T_range = np.arange(300, 3000)

for key, value in spc_dict.items():
    thermo = value.thermo
    # Get valid temperature range; some entries may not have the range
    try:
        Ts = np.arange(thermo.Tmin.value, thermo.Tmax.value)
    except AttributeError:
        Ts = default_T_range
    entropies, cps = np.zeros_like(Ts), np.zeros_like(Ts)
    for i, T in enumerate(Ts):
        entropies[i] = thermo.get_entropy(T)
        cps[i] = thermo.get_heat_capacity(T)
    if np.any(entropies < 0) or np.any(cps < 0):
        print(f'Error: Entropy or heat capacity has negative values: {value.to_chemkin()}:{key}')
        plot_thermo(Ts, thermo=thermo)

### 2.2 Heat capacity monotonically increasing
We expect to see most species' heat capacity has a monotonically increasing trend

In [16]:
default_T_range = np.arange(300, 3000)

for key, value in spc_dict.items():
    thermo = value.thermo
    try:
        Tmin = thermo.Tmin.value_si
    except AttributeError:
        Tmin = default_T_range.min()
    else:
        Tmin = max(default_T_range.min(), Tmin)
    try:
        Tmax = thermo.Tmax.value_si
    except AttributeError:
        Tmax = default_T_range.max()
    else:
        Tmax = min(default_T_range.max(), Tmax)

    local_extreme = []
    for poly, T_range in zip(['poly1', 'poly2'], ['L', 'H']):
        nasa_poly = getattr(value.thermo, poly)
        T_range = [Tmin, nasa_poly.Tmax.value_si] if T_range == 'L' \
                   else [nasa_poly.Tmin.value_si, Tmax]
        # Find the saddle point
        # 4*a5*T**3 + 3*a4*T**2 + 2*a3*T + a4
        derivative_coeffs = nasa_poly.coeffs[-3:0:-1] * np.array([4, 3, 2, 1])
        # Figure out maxima / minima
        sec_derivative_coeffs = derivative_coeffs[:-1] * np.array([3, 2, 1])
        # Find the roots
        roots = np.roots(derivative_coeffs)
        real_roots = []
        for i in range(roots.shape[0]):
            temp = roots[i]
            if np.isreal(temp) \
                    and temp < T_range[1] \
                    and temp > T_range[0]:
                real_roots.append(np.real(temp))

        for temp in real_roots:
            temp_poly = np.array([temp ** 2, temp, 1])
            second_de = np.dot(sec_derivative_coeffs, temp_poly)
            if second_de < 0:
                local_extreme.append(f'{temp:.2f} K, {thermo.get_heat_capacity(temp):.1f}J/mol/K (Maximum)')
            if second_de > 0:
                local_extreme.append(f'{temp:.2f} K, {thermo.get_heat_capacity(temp):.1f}J/mol/K (Minimum)')
            if second_de == 0:
                local_extreme.append(f'{temp:.2f} K, {thermo.get_heat_capacity(temp):.1f}J/mol/K (Unknown)')

    if local_extreme:
        print(f'Warning: Heat capacity is not monotonically increasing: {value.to_chemkin()}:{key}')
        print(f'{"; ".join(local_extreme)}')

2357.10 K, 400.0J/mol/K (Maximum)
2355.63 K, 37.2J/mol/K (Maximum)
2925.65 K, 20.8J/mol/K (Maximum); 1559.40 K, 20.8J/mol/K (Minimum); 506.00 K, 20.8J/mol/K (Maximum)
2925.65 K, 20.8J/mol/K (Maximum); 1559.40 K, 20.8J/mol/K (Minimum); 506.00 K, 20.8J/mol/K (Maximum)
409.05 K, 29.1J/mol/K (Minimum)
1904.69 K, 35.2J/mol/K (Maximum)
334.23 K, 29.1J/mol/K (Minimum)
2925.65 K, 20.8J/mol/K (Maximum); 1559.40 K, 20.8J/mol/K (Minimum); 506.00 K, 20.8J/mol/K (Maximum)
1789.37 K, 55.7J/mol/K (Maximum)
2924.56 K, 122.0J/mol/K (Minimum); 2688.40 K, 122.1J/mol/K (Maximum)
1864.09 K, 36.0J/mol/K (Maximum)
1776.56 K, 59.6J/mol/K (Maximum)
2812.15 K, 210.4J/mol/K (Maximum)
2222.35 K, 161.0J/mol/K (Maximum)
2469.29 K, 101.8J/mol/K (Maximum)
2027.84 K, 78.8J/mol/K (Maximum)
2913.64 K, 192.0J/mol/K (Maximum)
2881.10 K, 413.2J/mol/K (Maximum)
1710.65 K, 264.9J/mol/K (Maximum)
2842.33 K, 213.5J/mol/K (Maximum)
2785.02 K, 260.9J/mol/K (Maximum)
2865.10 K, 217.7J/mol/K (Maximum)
2816.14 K, 222.0J/mol/K (Maxi